
# 🔧 MolForge Tools — Smoke Test Notebook

Este notebook prueba rápidamente que el entorno **`molforge-tools`** funciona: `numpy`, `pandas`, `pyarrow` e **`rdkit`**, además de `ipywidgets`.

**Instrucciones**:
1. Abre este `.ipynb` en JupyterLab o VS Code.
2. Selecciona el kernel **Python (molforge-tools)**.
3. Ejecuta **Run All**. Si todo va bien, verás versiones, tablas y resultados sin errores.


In [1]:

import sys, platform
import numpy as np
import pandas as pd

# pyarrow
import pyarrow as pa
import pyarrow.parquet as pq

# rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors

# ipywidgets
import ipywidgets as widgets
from IPython.display import display, Markdown

print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())

print("numpy:", np.__version__)
print("pandas:", pd.__version__)
print("pyarrow:", pa.__version__)

# rdkit version is available via RDLogger or rdBase
from rdkit import rdBase
print("rdkit:", rdBase.rdkitVersion)

print("ipywidgets:", widgets.__version__)
display(Markdown("**✅ Importación correcta de todos los paquetes.**"))


Python: 3.14.0
Platform: Linux-6.14.0-33-generic-x86_64-with-glibc2.41
numpy: 2.3.3
pandas: 2.3.3
pyarrow: 21.0.0
rdkit: 2025.09.1
ipywidgets: 8.1.7


**✅ Importación correcta de todos los paquetes.**

In [2]:

# NumPy: una pequeña operación determinista
a = np.arange(1, 6, dtype=float)
b = np.linspace(0, 1, 5)
c = a @ b  # producto punto
print("a =", a)
print("b =", b)
print("a·b =", c)

# pandas: DataFrame simple
df = pd.DataFrame({
    "x": np.arange(5),
    "y": np.power(np.arange(5), 2),
    "txt": list("abcde")
})
display(df)

print("✅ NumPy y pandas funcionando.")


a = [1. 2. 3. 4. 5.]
b = [0.   0.25 0.5  0.75 1.  ]
a·b = 10.0


,x,y,txt
0,0,0,a
1,1,1,b
2,2,4,c
3,3,9,d
4,4,16,e


✅ NumPy y pandas funcionando.


In [3]:

# Crear una tabla Arrow desde pandas y hacer round-trip Parquet en memoria
table = pa.Table.from_pandas(df)
import io
buf = io.BytesIO()
pq.write_table(table, buf)
buf.seek(0)
table2 = pq.read_table(buf)
df2 = table2.to_pandas()
display(df2)

assert df2.equals(df), "El DataFrame tras el round-trip debe ser igual al original"
print("✅ PyArrow in-memory Parquet round-trip OK (sin tocar disco).")


,x,y,txt
0,0,0,a
1,1,1,b
2,2,4,c
3,3,9,d
4,4,16,e


✅ PyArrow in-memory Parquet round-trip OK (sin tocar disco).


In [4]:
from rdkit import Chem, RDLogger
from rdkit.Chem import Descriptors, rdFingerprintGenerator
import pandas as pd

# (opcional) silenciar avisos de RDKit
RDLogger.DisableLog('rdApp.warning')

# RDKit: molécula de ejemplo (Aspirina)
smiles = "CC(=O)OC1=CC=CC=C1C(=O)O"
mol = Chem.MolFromSmiles(smiles)
assert mol is not None, "RDKit no pudo parsear el SMILES"

# Propiedades básicas
mw = Descriptors.MolWt(mol)
can_smi = Chem.MolToSmiles(mol, canonical=True)

# Número de anillos (API moderna)
ring_info = mol.GetRingInfo()
ring_count = ring_info.NumRings()           # entero
atom_rings = ring_info.AtomRings()          # tuplas con índices de átomos en cada anillo

# Morgan fingerprint (ECFP4 = radio 2, 2048 bits) con el generador moderno
gen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)
fp = gen.GetFingerprint(mol)                # ExplicitBitVect
on_bits = list(fp.GetOnBits())

print("SMILES (input):", smiles)
print("SMILES (canonical):", can_smi)
print("MolWeight:", round(mw, 4))
print("Num rings:", ring_count)
print("ECFP4 bits on:", len(on_bits), "ejemplo de índices:", on_bits[:10])

# DataFrame resumen
df_props = pd.DataFrame([{
    "smiles_in": smiles,
    "smiles_canonical": can_smi,
    "mol_weight": mw,
    "num_rings": ring_count,
    "ecfp4_bits_on": len(on_bits),
    "first_ring_example_atom_ids": list(atom_rings[0]) if ring_count > 0 else None
}])
display(df_props)

print("✅ RDKit básico funcionando (parseo, propiedades, fingerprint).")

SMILES (input): CC(=O)OC1=CC=CC=C1C(=O)O
SMILES (canonical): CC(=O)Oc1ccccc1C(=O)O
MolWeight: 180.159
Num rings: 1
ECFP4 bits on: 24 ejemplo de índices: [389, 456, 650, 695, 807, 909, 1017, 1035, 1047, 1057]


,smiles_in,smiles_canonical,mol_weight,num_rings,ecfp4_bits_on,first_ring_example_atom_ids
0,CC(=O)OC1=CC=CC=C1C(=O)O,CC(=O)Oc1ccccc1C(=O)O,180.159,1,24,"[4, 9, 8, 7, 6, 5]"


✅ RDKit básico funcionando (parseo, propiedades, fingerprint).


In [5]:

# Widgets: slider que muestra un cálculo y una tabla derivada
slider = widgets.IntSlider(value=3, min=0, max=10, step=1, description="n:")
out = widgets.Output()

def on_change(change):
    if change["name"] == "value":
        n = change["new"]
        with out:
            out.clear_output()
            arr = np.arange(n+1)
            dfw = pd.DataFrame({"k": arr, "k^2": arr**2})
            display(Markdown(f"**n = {n} → n^2 = {n*n}**"))
            display(dfw)

slider.observe(on_change)
display(slider)
display(out)

# Disparar una primera salida
slider.value = slider.value  # fuerza el callback

print("✅ ipywidgets renderizado (si no ves el slider, comprueba que ipywidgets está habilitado).")


IntSlider(value=3, description='n:', max=10)

Output()

✅ ipywidgets renderizado (si no ves el slider, comprueba que ipywidgets está habilitado).
